In [1]:
import pandas as pd
import numpy as np
import os
import torch
import time
import json

from datasets import Dataset
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)

os.environ["TOKENIZERS_PARALLELISM"] = "false"

2023-10-15 10:20:17.132636: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-15 10:20:18.017643: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-10-15 10:20:18.017759: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-10-15 10:20:18.017770: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [2]:
df = pd.read_excel('../dataset/ije_sa/ije_sa.xlsx')
train_data = pd.read_excel('../dataset/ije_sa/train_set.xlsx')
val_data = pd.read_excel('../dataset/ije_sa/validation_set.xlsx')
test_data = pd.read_excel('../dataset/ije_sa/test_set.xlsx')

# Length train, val, and test
print("Train: ",len(train_data))
print("Val: ",len(val_data))
print("Test: ",len(test_data))

tags = np.unique(df['label'])
num_labels = len(tags)
max_length = 128
label2id = {t: i for i, t in enumerate(tags)}
id2label = {i: t for i, t in enumerate(tags)}

Train:  1350
Val:  116
Test:  463


In [3]:
def model_init(model_name):
    global tokenizer
    global data_collator
    global tr_model

    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=num_labels,
        id2label=id2label,
        label2id=label2id
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=128)
    
    return model, tokenizer

In [4]:
def tokenize_function(examples):
    # process the input sequence
    tokenized_input = tokenizer(examples["tweet"], 
                                truncation=True, 
                                padding='max_length', 
                                max_length=max_length)
    # process the labels
    tokenized_input['label'] = [label2id[lb] for lb in examples['label']]
    
    return tokenized_input

In [5]:
def preprocessing():
    X_train = Dataset.from_pandas(train_data)
    X_val = Dataset.from_pandas(val_data)
    X_test = Dataset.from_pandas(test_data)
    
    tokenized_train_data = X_train.map(tokenize_function, batched=True)
    tokenized_val_data = X_val.map(tokenize_function, batched=True)
    tokenized_test_data = X_test.map(tokenize_function, batched=True)
    
    return tokenized_train_data, tokenized_val_data, tokenized_test_data

In [6]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    
    true_labels = [tags[l] for l in labels] 
    true_predictions = [tags[pr] for pr in pred]

    report = classification_report(true_labels, true_predictions, digits=4)
    acc = accuracy_score(y_true=true_labels, y_pred=true_predictions)
    rec = recall_score(y_true=true_labels, y_pred=true_predictions, average='macro')
    prec = precision_score(y_true=true_labels, y_pred=true_predictions, average='macro')
    f1 = f1_score(y_true=true_labels, y_pred=true_predictions, average='macro', zero_division=1.0)

    print("Classification Report:\n{}".format(report))
    return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1}

In [7]:
def train_model(model_name, output_dir, 
                learning_rate, train_batch_size, 
                eval_batch_size, num_epochs, weight_decay):
    model, tokenizer = model_init(model_name)
    train_tokenized, val_tokenized, test_tokenized = preprocessing()
    
    training_args = TrainingArguments(
        output_dir=output_dir,
        logging_strategy="epoch",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        save_total_limit = 1,
        learning_rate=learning_rate,
        num_train_epochs=num_epochs,
        per_device_train_batch_size=train_batch_size,
        per_device_eval_batch_size=eval_batch_size,
        weight_decay=weight_decay,
        #push_to_hub=True,
        metric_for_best_model = "f1",
        load_best_model_at_end=True
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_tokenized,
        eval_dataset=val_tokenized,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )
    trainer.train()
    trainer.save_model(output_dir)
    #trainer.push_to_hub(commit_message="Training complete")

In [8]:
def predict_model(model_name):
    model, tokenizer = model_init(model_name)
    train_tokenized, val_tokenized, test_tokenized = preprocessing()

    trainer = Trainer(
        model=model,
        compute_metrics=compute_metrics,
    )

    # trainer.predict(test_tokenized)
    # trainer.push_to_hub(commit_message="Test complete")
    print(trainer.predict(test_tokenized))

In [9]:
def main(model_name, output_dir, best_params):
    start = time.time()
     # load json file containing best params
    best_params = best_params

    with open(best_params, 'r') as js:
        data = json.load(js)

    print(data)

    # define best params
    learning_rate = data['learning_rate']
    train_batch_size = data['per_device_train_batch_size']
    eval_batch_size = data['per_device_eval_batch_size']
    weight_decay = data['weight_decay']
    num_epochs = data['num_epochs']

    # training
    train_model(model_name=model_name, 
                output_dir=output_dir,
                learning_rate=learning_rate,
                train_batch_size=train_batch_size,
                eval_batch_size=eval_batch_size,
                num_epochs=num_epochs,
                weight_decay=weight_decay)

    print('Training finished!')
    
    # prediction
    predict_model(model_name=f'{output_dir}')
    print('Prediction finished!')

    
    end = time.time()
    exec_time = (end - start) / 60
    print(f'Total time: {exec_time} minutes')

In [10]:
#1 fathan/indojave-codemixed-bert-base
main(
    model_name = 'fathan/indojave-codemixed-bert-base',
    output_dir = 'IJE_SA_MODEL/ije-sa-ft-indojave-cm-bert',
    best_params = 'best_parameters/indojave-cm-bert.json'
)

{'num_epochs': 8, 'learning_rate': 5e-05, 'per_device_train_batch_size': 16, 'per_device_eval_batch_size': 8, 'weight_decay': 0.0007450441482348415}


Some weights of the model checkpoint at fathan/indojave-codemixed-bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at fathan/in

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

...

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweet, clean_tweet. If tweet, clean_tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 463
  Batch size = 32
/home/jupyter-145230403/.conda/envs/conda_env_2/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dime

Classification Report:
              precision    recall  f1-score   support

    Negative     0.8961    0.9650    0.9293       143
     Neutral     0.9869    0.9264    0.9557       163
    Positive     0.9423    0.9363    0.9393       157

    accuracy                         0.9417       463
   macro avg     0.9418    0.9426    0.9414       463
weighted avg     0.9437    0.9417    0.9420       463

PredictionOutput(predictions=array([[ 4.514627  , -3.9299123 , -1.1667176 ],
       [ 4.871687  , -3.0491009 , -2.2938764 ],
       [ 4.8515363 , -3.373619  , -1.9560899 ],
       ...,
       [-2.054923  ,  2.6831512 ,  0.21166822],
       [ 4.772097  , -3.1108046 , -2.0322428 ],
       [-3.312058  , -0.2243209 ,  3.7559423 ]], dtype=float32), label_ids=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2,

In [11]:
#2 fathan/indojave-codemixed-roberta-base

main(
    model_name = 'fathan/indojave-codemixed-roberta-base',
    output_dir = 'IJE_SA_MODEL/ije-sa-ft-indojave-cm-roberta',
    best_params = 'best_parameters/indojave-cm-roberta.json'
)

{'num_epochs': 6, 'learning_rate': 5e-05, 'per_device_train_batch_size': 8, 'per_device_eval_batch_size': 8, 'weight_decay': 0.0006240436334981423}


loading configuration file config.json from cache at /home/jupyter-145230403/.cache/huggingface/hub/models--fathan--indojave-codemixed-roberta-base/snapshots/487f71ebf847f059c180212fc636a7efbb0cbf70/config.json
Model config RobertaConfig {
  "_name_or_path": "fathan/indojave-codemixed-roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "Negative",
    "1": "Neutral",
    "2": "Positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "Negative": 0,
    "Neutral": 1,
    "Positive": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transfor

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

...

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: tweet, clean_tweet. If tweet, clean_tweet are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 463
  Batch size = 32
/home/jupyter-145230403/.conda/envs/conda_env_2/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather alon

Classification Report:
              precision    recall  f1-score   support

    Negative     0.8792    0.9161    0.8973       143
     Neutral     0.9281    0.9509    0.9394       163
    Positive     0.9252    0.8662    0.8947       157

    accuracy                         0.9114       463
   macro avg     0.9108    0.9111    0.9105       463
weighted avg     0.9120    0.9114    0.9112       463

PredictionOutput(predictions=array([[ 4.9040475 , -2.8733916 , -2.01359   ],
       [ 4.9321246 , -2.8725445 , -2.077031  ],
       [ 4.9500737 , -2.7731822 , -2.2550828 ],
       ...,
       [-2.851556  ,  4.298276  , -1.5379208 ],
       [ 4.6373005 , -2.0078518 , -2.6047714 ],
       [-3.2134812 , -0.44903287,  4.296734  ]], dtype=float32), label_ids=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2,

In [13]:
#3 fathan/indojave-codemixed-indobert-base
main(
    model_name = 'fathan/indojave-codemixed-indobert-base',
    output_dir = 'IJE_SA_MODEL/ije-sa-ft-indojave-cm-indobert',
    best_params = 'best_parameters/indojave-cm-indobert.json'
)

{'num_epochs': 5, 'learning_rate': 5e-05, 'per_device_train_batch_size': 8, 'per_device_eval_batch_size': 8, 'weight_decay': 0.00010054838958197635}


loading configuration file config.json from cache at /home/jupyter-145230403/.cache/huggingface/hub/models--fathan--indojave-codemixed-indobert-base/snapshots/4a4c322f69fa8ae518250f539dc4b25ba2448885/config.json
Model config BertConfig {
  "_name_or_path": "fathan/indojave-codemixed-indobert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_ids": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "Negative",
    "1": "Neutral",
    "2": "Positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "Negative": 0,
    "Neutral": 1,
    "Positive": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "floa

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

...

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweet, clean_tweet. If tweet, clean_tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 463
  Batch size = 32
/home/jupyter-145230403/.conda/envs/conda_env_2/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dime

Classification Report:
              precision    recall  f1-score   support

    Negative     0.9211    0.9790    0.9492       143
     Neutral     0.9936    0.9571    0.9750       163
    Positive     0.9675    0.9490    0.9582       157

    accuracy                         0.9611       463
   macro avg     0.9607    0.9617    0.9608       463
weighted avg     0.9624    0.9611    0.9613       463

PredictionOutput(predictions=array([[ 4.9390697, -3.7292545, -1.4951298],
       [ 5.1308875, -3.380357 , -2.1732352],
       [ 5.0868425, -3.2506335, -2.1547565],
       ...,
       [-2.2854433, -1.2984508,  4.1714864],
       [ 4.988334 , -3.5435681, -1.9317676],
       [-2.1331456, -2.3069801,  5.1612396]], dtype=float32), label_ids=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0,

In [12]:
#4 fathan/indojave-codemixed-indobertweet-base
main(
    model_name = 'fathan/indojave-codemixed-indobertweet-base',
    output_dir = 'IJE_SA_MODEL/ije-sa-ft-indojave-cm-indobertweet',
    best_params = 'best_parameters/indojave-cm-indobertweet.json'
)

{'num_epochs': 5, 'learning_rate': 0.0003, 'per_device_train_batch_size': 16, 'per_device_eval_batch_size': 32, 'weight_decay': 0.003234558453531053}


loading configuration file config.json from cache at /home/jupyter-145230403/.cache/huggingface/hub/models--fathan--indojave-codemixed-indobertweet-base/snapshots/513d49f87b1bf3519a35fc5ed104e6a39ab97fae/config.json
Model config BertConfig {
  "_name_or_path": "fathan/indojave-codemixed-indobertweet-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_ids": 0,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "Negative",
    "1": "Neutral",
    "2": "Positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "Negative": 0,
    "Neutral": 1,
    "Positive": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "position_embeddin

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

...

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweet, clean_tweet. If tweet, clean_tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 463
  Batch size = 32
/home/jupyter-145230403/.conda/envs/conda_env_2/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dime

Classification Report:
              precision    recall  f1-score   support

    Negative     0.9205    0.9720    0.9456       143
     Neutral     0.9873    0.9571    0.9720       163
    Positive     0.9416    0.9236    0.9325       157

    accuracy                         0.9503       463
   macro avg     0.9498    0.9509    0.9500       463
weighted avg     0.9512    0.9503    0.9504       463

PredictionOutput(predictions=array([[ 4.420567 , -3.4320877, -1.1812421],
       [ 4.558454 , -2.7492235, -2.0057154],
       [ 4.575959 , -2.9644396, -1.8043576],
       ...,
       [ 0.573043 , -3.0729578,  3.0017962],
       [ 4.56535  , -2.8024833, -1.8594264],
       [-0.7342177, -2.7236228,  4.0794168]], dtype=float32), label_ids=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0,

In [14]:
#5 indolem/indobert-base-uncased

main(
    model_name = 'indolem/indobert-base-uncased',
    output_dir = 'IJE_SA_MODEL/ije-sa-ft-indobert',
    best_params = 'best_parameters/indobert.json'
)

{'num_epochs': 9, 'learning_rate': 5e-05, 'per_device_train_batch_size': 16, 'per_device_eval_batch_size': 64, 'weight_decay': 0.0001287688045739695}


loading configuration file config.json from cache at /home/jupyter-145230403/.cache/huggingface/hub/models--indolem--indobert-base-uncased/snapshots/7ccb3cd0f5b08ffbaa465aade22328e8600e23eb/config.json
Model config BertConfig {
  "_name_or_path": "indolem/indobert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_ids": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "Negative",
    "1": "Neutral",
    "2": "Positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "Negative": 0,
    "Neutral": 1,
    "Positive": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "typ

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

...

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweet, clean_tweet. If tweet, clean_tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 463
  Batch size = 32
/home/jupyter-145230403/.conda/envs/conda_env_2/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dime

Classification Report:
              precision    recall  f1-score   support

    Negative     0.8609    0.9091    0.8844       143
     Neutral     0.9671    0.9018    0.9333       163
    Positive     0.9187    0.9363    0.9274       157

    accuracy                         0.9158       463
   macro avg     0.9156    0.9157    0.9150       463
weighted avg     0.9179    0.9158    0.9162       463

PredictionOutput(predictions=array([[ 4.528869  , -2.0435696 , -2.4441047 ],
       [ 4.529774  , -2.8338451 , -1.7457023 ],
       [ 4.354292  , -2.575475  , -1.7396991 ],
       ...,
       [-2.5459285 ,  3.4859998 , -0.0123857 ],
       [ 3.9293919 , -3.2360063 , -0.8946097 ],
       [-1.4233307 , -0.19207735,  1.9057432 ]], dtype=float32), label_ids=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2,

In [15]:
#6 indolem/indobertweet-base-uncased

main(
    model_name = 'indolem/indobertweet-base-uncased',
    output_dir = 'IJE_SA_MODEL/ije-sa-ft-indobertweet',
    best_params = 'best_parameters/indobertweet.json'
)

{'num_epochs': 9, 'learning_rate': 3e-05, 'per_device_train_batch_size': 16, 'per_device_eval_batch_size': 16, 'weight_decay': 0.0002994715145894433}


loading configuration file config.json from cache at /home/jupyter-145230403/.cache/huggingface/hub/models--indolem--indobertweet-base-uncased/snapshots/32e28c05b47e33b6675d2670a1745c50a65e987a/config.json
Model config BertConfig {
  "_name_or_path": "indolem/indobertweet-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_ids": 0,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "Negative",
    "1": "Neutral",
    "2": "Positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "Negative": 0,
    "Neutral": 1,
    "Positive": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

...

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweet, clean_tweet. If tweet, clean_tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 463
  Batch size = 32
/home/jupyter-145230403/.conda/envs/conda_env_2/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dime

Classification Report:
              precision    recall  f1-score   support

    Negative     0.8903    0.9650    0.9262       143
     Neutral     0.9934    0.9202    0.9554       163
    Positive     0.9427    0.9427    0.9427       157

    accuracy                         0.9417       463
   macro avg     0.9421    0.9427    0.9414       463
weighted avg     0.9444    0.9417    0.9421       463

PredictionOutput(predictions=array([[ 4.273509  , -2.5944867 , -2.377041  ],
       [ 4.820379  , -3.3244283 , -2.5188272 ],
       [ 4.5697417 , -3.130979  , -2.3377383 ],
       ...,
       [-2.565445  ,  1.1667743 ,  2.3371422 ],
       [ 4.6501203 , -3.131346  , -2.320866  ],
       [-2.52371   ,  0.02041547,  3.0547304 ]], dtype=float32), label_ids=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2,

In [16]:
#7 bert-base-multilingual-uncased

main(
    model_name = 'bert-base-multilingual-uncased',
    output_dir = 'IJE_SA_MODEL/ije-sa-ft-mbert',
    best_params = 'best_parameters/mbert.json'
)

{'num_epochs': 5, 'learning_rate': 0.0001, 'per_device_train_batch_size': 8, 'per_device_eval_batch_size': 32, 'weight_decay': 9.512565513083272e-05}


loading configuration file config.json from cache at /home/jupyter-145230403/.cache/huggingface/hub/models--bert-base-multilingual-uncased/snapshots/3da6b6aad5111664db74322f2158b7f93e09a717/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "Negative",
    "1": "Neutral",
    "2": "Positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "Negative": 0,
    "Neutral": 1,
    "Positive": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

...

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweet, clean_tweet. If tweet, clean_tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 463
  Batch size = 32
/home/jupyter-145230403/.conda/envs/conda_env_2/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dime

Classification Report:
              precision    recall  f1-score   support

    Negative     0.6888    0.9441    0.7965       143
     Neutral     0.9846    0.7853    0.8737       163
    Positive     0.8978    0.7834    0.8367       157

    accuracy                         0.8337       463
   macro avg     0.8571    0.8376    0.8356       463
weighted avg     0.8638    0.8337    0.8373       463

PredictionOutput(predictions=array([[ 2.9048173 , -2.5359194 , -0.8562975 ],
       [ 2.8924458 , -2.4742615 , -0.9189515 ],
       [ 2.8946009 , -2.4517972 , -0.9423044 ],
       ...,
       [ 0.05830768, -2.601041  ,  2.420907  ],
       [ 2.9236724 , -2.5925229 , -0.7496682 ],
       [ 0.6142814 , -2.726361  ,  2.0233245 ]], dtype=float32), label_ids=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2,

In [18]:
#8 distilbert-base-multilingual-cased

main(
    model_name = 'distilbert-base-multilingual-cased',
    output_dir = 'IJE_SA_MODEL/ije-sa-ft-distilmbert',
    best_params = 'best_parameters/distilmbert.json'
)

{'num_epochs': 6, 'learning_rate': 0.0001, 'per_device_train_batch_size': 16, 'per_device_eval_batch_size': 32, 'weight_decay': 0.0012506415538934051}


loading configuration file config.json from cache at /home/jupyter-145230403/.cache/huggingface/hub/models--distilbert-base-multilingual-cased/snapshots/9e90d6dd84b6a2e4d65e4d751baed6cd56578fd3/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-multilingual-cased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "Negative",
    "1": "Neutral",
    "2": "Positive"
  },
  "initializer_range": 0.02,
  "label2id": {
    "Negative": 0,
    "Neutral": 1,
    "Positive": 2
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.26.1",
  "vocab_size": 119547
}

loading weights file model.safetensors from ca

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

...

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: tweet, clean_tweet. If tweet, clean_tweet are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 463
  Batch size = 32
/home/jupyter-145230403/.conda/envs/conda_env_2/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gathe

Classification Report:
              precision    recall  f1-score   support

    Negative     0.7095    0.8881    0.7888       143
     Neutral     0.9236    0.8160    0.8664       163
    Positive     0.8643    0.7707    0.8148       157

    accuracy                         0.8229       463
   macro avg     0.8325    0.8249    0.8234       463
weighted avg     0.8374    0.8229    0.8250       463

PredictionOutput(predictions=array([[ 3.3847132 , -2.7222726 , -1.4551932 ],
       [ 3.699325  , -2.6873791 , -1.8124403 ],
       [ 3.662219  , -2.780971  , -1.6497129 ],
       ...,
       [ 2.317116  , -1.0851647 , -1.5622102 ],
       [ 2.8477736 , -3.3111115 , -0.42939085],
       [-1.8452371 , -1.3140638 ,  3.24251   ]], dtype=float32), label_ids=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2,

In [17]:
#9 xlm-roberta-base

main(
    model_name = 'xlm-roberta-base',
    output_dir = 'IJE_SA_MODEL/ije-sa-ft-xlm-roberta',
    best_params = 'best_parameters/xlm-roberta.json'
)

{'num_epochs': 9, 'learning_rate': 0.0001, 'per_device_train_batch_size': 64, 'per_device_eval_batch_size': 64, 'weight_decay': 0.0011724870810857187}


loading configuration file config.json from cache at /home/jupyter-145230403/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/77de1f7a7e5e737aead1cd880979d4f1b3af6668/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "Negative",
    "1": "Neutral",
    "2": "Positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "Negative": 0,
    "Neutral": 1,
    "Positive": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_s

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

...

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: tweet, clean_tweet. If tweet, clean_tweet are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 463
  Batch size = 32
/home/jupyter-145230403/.conda/envs/conda_env_2/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gathe

Classification Report:
              precision    recall  f1-score   support

    Negative     0.8012    0.9580    0.8726       143
     Neutral     0.9452    0.8466    0.8932       163
    Positive     0.9315    0.8662    0.8977       157

    accuracy                         0.8877       463
   macro avg     0.8926    0.8903    0.8878       463
weighted avg     0.8961    0.8877    0.8884       463

PredictionOutput(predictions=array([[ 1.439427  , -2.0475745 ,  0.32817125],
       [ 2.6250486 , -2.1165354 , -0.85414433],
       [ 2.3578537 , -1.1590322 , -1.4477365 ],
       ...,
       [-1.6875925 ,  2.3683002 , -1.2940696 ],
       [ 2.693726  , -1.9362189 , -0.97334987],
       [-1.3465405 , -0.7248404 ,  1.3096148 ]], dtype=float32), label_ids=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2,